In [52]:

import mne
from mne.preprocessing import ICA


import mtrf
from mtrf.model import TRF
from mtrf.stats import crossval, nested_crossval
import eelbrain as eel

import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.io import wavfile, loadmat, savemat
from scipy.signal import hilbert

from collections import Counter

import sys
sys.path.append('../utils')
from pp_utils import *
from plot_utils import *
from mTRF_utils import *
from midi_utils import *
from update_sub_lists import*
from ERP_utils import *

import pickle
import os

In [53]:
task = 'motor'
eeg_dir = '/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/data_preprocessed_30Hz'
mTRF_path = f'/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/analysis_{task}/{task}_mTRF_data_30Hz'


In [54]:
#subjects_to_process = find_existing_subjects(eeg_dir)
#subjects_to_process = ['11']
good_listen_subjects, good_motor_subjects,good_error_subjects,musicians,nonmusicians = load_subject_lists_LME()
subjects_to_process = good_motor_subjects
n_subs = len(subjects_to_process)
print('processing subjects:', subjects_to_process)


processing subjects: ['01', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '18', '19', '20']


In [55]:
periods = ['pre', 'post']
#features = 'AM' #AM or onsets

overwrite = False
n_segments = 10

#OPENING SUPPORTING FILES
#######################################################

with open('../utils/ch_names.pkl', 'rb') as file:
    ch_names_all = pickle.load(file)
ch_names_64 = ch_names_all[0:64]
ch_names_72 = ch_names_all[0:72]

fs = 128
tmin, tmax =-0.5, 0.5  # range of time lag
#regularizations= [0.0001, 0.01, 1, 100, 10000, 1000000]


In [56]:

for subject in subjects_to_process:
    for period in periods:
        eeg_path = os.path.join(eeg_dir, f'{subject}/eeg_{task}_{period}_{subject}.mat')
        data = loadmat(eeg_path)
        eeg = data['trial_data']
        events_sv = data['events']

        events_arr = make_raw_events(events_sv)
        events_keystrokes = clean_triggers(events_arr[events_arr[:, 2]==2])
        onset_indices = events_keystrokes[:,0]
        onsets_sv = np.zeros_like(events_sv[0])
        onsets_sv[onset_indices] = 1

        eeg_segments = segment(eeg.T, n_segments)
        onset_segments = segment(onsets_sv, n_segments) 

        #training TRF model
        fwd_trf = TRF(direction=1)


        # Perform nested cross-validation
        #r_fwd, lambdas = nested_crossval(fwd_trf, onset_segments, eeg_segments, fs, tmin, tmax, regularizations)

        # Select the best lambda aka the one that was chosen the most frequently
        lambda_counts = Counter(lambdas)
        most_frequent_lambda = lambda_counts.most_common(1)[0][0]


        #r_fwd, lambdas = nested_crossval(fwd_trf, stimulus, eeg_segments, fs, tmin, tmax, regularizations) #skipping for now 
        #regularization = 1000 # regularization parameter lambda
        fwd_trf.train(onset_segments, eeg_segments, fs, tmin, tmax, regularization = 1000)
        
        
        if not os.path.exists(mTRF_path):
            os.mkdir(mTRF_path)
        savemat(f'{mTRF_path}/mTRFweights_{task}_{period}_{subject}.mat', {'weights': fwd_trf.weights, 'r': r_fwd})

In [57]:
"""fig, ax = plt.subplots(2)
fwd_trf.plot(feature=0, axes=ax[0], show=True)"""

'fig, ax = plt.subplots(2)\nfwd_trf.plot(feature=0, axes=ax[0], show=True)'